In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver chromium-browser
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium beautifulsoup4 pandas openpyxl lxml requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import re
from datetime import datetime
import requests
from bs4 import BeautifulSoup

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

# БЛОК 2
настройка браузера

In [ ]:
def setup_browser():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--remote-debugging-port=9222')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36')
    chrome_options.binary_location = '/usr/bin/chromium-browser'

    driver = webdriver.Chrome(options=chrome_options)
    return driver

# БЛОК 3
Прокрутка страниц

In [ ]:
def load_all_articles(driver, target_articles=10100):

    driver.get("https://www.cnbc.com/search/?query=nvidia&qsearchterm=nvidia")
    time.sleep(10)

    newest_button = driver.find_element(By.CSS_SELECTOR, "#sortdate")
    newest_button.click()

    last_count = 0
    scroll_attempts = 0
    max_scrolls = target_articles*2
    no_change_count = 0

    print("Цель: ", target_articles, "статей")

    while scroll_attempts < max_scrolls and no_change_count < 20:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)

        current_articles = driver.find_elements(By.CSS_SELECTOR, '.SearchResult-searchResultContent')
        current_count = len(current_articles)
        scroll_attempts += 1

        print("Прокрутка ", scroll_attempts," : ", current_count, " статей")

        if current_count == last_count:
            no_change_count += 1
            print("Количество не изменилось", no_change_count, "/10")
        else:
            no_change_count = 0

        last_count = current_count


        if current_count >= target_articles:
            print("Достигнута цель в ", target_articles, "статей")
            break

    print("Загрузка завершена. всего статей: ", last_count)
    return last_count

# БЛОК 4
Захват данных со страницы

In [ ]:
def parse_search_page(driver):

    articles_data = []

    time.sleep(3)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')


    search_container = soup.find('div', id='searchcontainer')
    if not search_container:
        print("searchcontainer не найден ")
        return articles_data


    article_cards = search_container.find_all('div', class_='SearchResult-searchResult')
    print("Найдено ", len(article_cards), "карточек статей")

    for i, card in enumerate(article_cards):
        try:
            #заголовок
            title = "Не найден"
            title_container = card.find('div', class_='SearchResult-searchResultTitle')
            if title_container:
                link_element = title_container.find('a')
                if link_element:
                    title = link_element.get_text(strip=True, separator = ' ')

                    url = "Не найдена"
                    if link_element.get('href'):
                        url = link_element['href']
                        if url.startswith('/'):
                            url = "https://www.cnbc.com"+url

            if title == "Не найден":
                continue
            #подзаголовок
            preview = "Не найдено"
            preview_element = card.find('p', class_='SearchResult-searchResultPreview')
            if preview_element:
                preview = preview_element.get_text(strip=True, separator = ' ')

            author = "Не указан"
            author_element = card.find('a', class_='SearchResult-author')
            if author_element:
                author = author_element.get_text(strip=True)

            publish_date = "Не указана"
            date_element = card.find('span', class_='SearchResult-publishedDate')
            if date_element:
                publish_date = date_element.get_text(strip=True)
            #раздел
            topic = "Не указана"
            topic_element = card.find('div', class_='SearchResult-searchResultEyebrow')
            if topic_element:
                topic = topic_element.get_text(strip=True)

            articles_data.append({
                'topic': topic,
                'title': title,
                'preview': preview,
                'author': author,
                'publish_date': publish_date,
                'url': url
            })

        except Exception as e:
            print("Ошибка при обработке карточки ", i + 1, e)
            continue

    print("Парсинг завершен. ", len(articles_data), " уникальных статей")
    return articles_data

# БЛОК 5
Парсинг текста статей



In [ ]:
def parse_article_details(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        response = requests.get(url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        full_content = "Текст не найден"
        content_elements = soup.find_all('div', class_=['ArticleBody-articleBody', 'group'])
        if not content_elements:
            content_elements = soup.find_all('div', attrs={'data-testid': 'article-body'})


        if content_elements:
            paragraphs = []
            for content_element in content_elements:
                paragraph_elements = content_element.find_all('p')
                for p in paragraph_elements:
                    text = p.get_text(strip=True, separator = ' ')
                    if text and len(text) > 20:
                        paragraphs.append(text)

            if paragraphs:
                full_content = ' '.join(paragraphs)

        else:
            content_elements = soup.find("div", class_= "ClipPlayer-clipPlayerIntro")
            content_element = content_elements.find('div', class_=["ClipPlayer-clipPlayerIntroSummary", "ClipPlayer-investingClubClipPlayerIntroSummary"])
            text = content_element.get_text(strip=True, separator = ' ')
            full_content = text

        return full_content

    except Exception as e:
        print("Ошибка при обработке статьи", url,":", e)
        return "Текст не найден"

# БЛОК 6
Основной процесс парсинга

In [ ]:
def main_parsing_process():
    driver = setup_browser()

    try:
        total_elements = load_all_articles(driver, 10100)

        articles_data = parse_search_page(driver)

        if not articles_data:
            print("Не найдено релевантных статей")
            return None

        final_data = []

        for i, article in enumerate(articles_data):

            details = parse_article_details(
                article['url']
            )

            complete_article = {
                'publish_date': article['publish_date'],
                'title': article['title'],
                'content': details,
                'author': article['author'],
                'topic': article['topic'],
                'preview': article['preview'],
                'url': article['url']
            }

            final_data.append(complete_article)
            time.sleep(1)

            if (i + 1) % 500 == 0:
                print("Обработано: ", i + 1, "статей")
                temp_df = pd.DataFrame(final_data)
                temp_filename = 'nvidia_news_progress_' + str(i+1) + '.csv'
                temp_df.to_csv(temp_filename, index=False)

        if final_data:
            df = pd.DataFrame(final_data)

            csv_filename = 'nvidia_news_final.csv'
            df.to_csv(csv_filename, index=False)

            excel_filename = 'nvidia_news_final.xlsx'
            df.to_excel(excel_filename, index=False)

            print("Конец парсинга")

            print("Всего: ", len(df),"новостей")
            print("CSV: ", csv_filename)
            print("Excel: ", excel_filename)

            return df

    except Exception as e:
        print("Критическая ошибка: ", e)
        import traceback
        traceback.print_exc()
        return None
    finally:
        driver.quit()
        print("Браузер закрыт")

# БЛОК 7
Финальный датасет

In [1]:
final_dataset = main_parsing_process()

def show_results(dataset):

    if dataset is not None:
        print("Размер: ", dataset.shape[0], "строк", dataset.shape[1], "колонок")
        print("Дата: ",len(dataset[dataset['publish_date'] != 'Не указана'])/len(dataset))
        print("Автор: ", len(dataset[dataset['author'] != 'Не указан'])/len(dataset))
        print("Текст: ", len(dataset[dataset['content'] != 'Текст не найден'])/len(dataset))
        print("Тема: ", len(dataset[dataset['topic'] != 'Не указана'])/len(dataset))
        print("Превью: ", len(dataset[dataset['preview'] != 'Не найдено'])/len(dataset))

if final_dataset is not None:
    show_results(final_dataset)

NameError: name 'main_parsing_process' is not defined

In [ ]:
def parse_complex_date(date_str):

    format = '%m/%d/%Y %I:%M:%S %p'
    try:
        return datetime.strptime(date_str, format)
    except ValueError:
        return None

final_dataset['publish_date'] = final_dataset['publish_date'].apply(parse_complex_date)

In [ ]:
final_dataset

,publish_date,title,content,author,topic,preview,url
0,2025-11-10 00:54:19,Here are 4 things that could go right for stoc...,Текст не найден,Jim Cramer,Investing Club Jim’s Columns,Go back to early April. Take a look at those d...,https://www.cnbc.com/2025/11/09/here-are-4-thi...
1,2025-11-09 03:58:08,Global week ahead: AI wobble casts shadow over...,Текст не найден,Leonie Kidd,Global Week Ahead,The city of Lisbon in Portugal turns to tech n...,https://www.cnbc.com/2025/11/09/global-week-ah...
2,2025-11-08 17:17:39,Week in review: The Nasdaq’s worst week since ...,Текст не найден,Morgan Chittum,CNBC Investing Club: Analysis,It was a terrible start to November on Wall St...,https://www.cnbc.com/2025/11/08/the-nasdaqs-wo...
3,2025-11-07 22:18:57,Buy the weakness in stocks like Tesla and Pala...,Текст не найден,Pia Singh,PRO: Power Check,Investors can buy the dip on some key tech sto...,https://www.cnbc.com/2025/11/07/buy-the-weakne...
4,2025-11-07 21:34:24,Palantir CEO Karp twice slams short sellers as...,Текст не найден,Samantha Subin,Technology,With Palantir’s stock plummeting more than 11%...,https://www.cnbc.com/2025/11/07/palantir-ceo-a...
...,...,...,...,...,...,...,...
95,2025-10-31 11:58:41,"Trump-Xi summit sounds like a ‘huge success’, ...",Текст не найден,Не указан,Squawk Box Europe,Nvidia CEO Jensen Huang discusses his meeting ...,https://www.cnbc.com/video/2025/10/31/trump-xi...
96,2025-10-31 11:51:43,Cloud revenue is sending Amazon shares soaring...,Текст не найден,Lisa Kailai Han,Pro: Analyst Calls,Amazon is rocketing higher after it posted a t...,https://www.cnbc.com/2025/10/31/cloud-revenue-...
97,2025-10-31 09:53:52,Nvidia CEO Jensen Huang says AI is in a ‘virtu...,Текст не найден,Tasmin Lockwood,Technology,Nvidia CEO Jensen Huang said Friday that artif...,https://www.cnbc.com/2025/10/31/nvidia-ceo-jen...
98,2025-10-31 08:57:03,Nvidia CEO: Samsung are excellent in their fie...,Текст не найден,Не указан,Squawk Box Europe,Nvidia CEO Jensen Huang tells CNBC the chip gi...,https://www.cnbc.com/video/2025/10/31/nvidia-c...
